In [ ]:
!pip install fast_ml sweetviz  --quiet

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#!pip install fast_ml 

from fast_ml.utilities import display_all
from fast_ml.feature_selection import get_duplicate_features

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
np.random.seed = 42

from sklearn.model_selection import train_test_split


#!pip install sweetviz

import sweetviz
from sklearn.preprocessing import RobustScaler
from sklearn import preprocessing
from fast_ml.feature_selection import get_constant_features

from sklearn.ensemble import RandomForestRegressor

In [ ]:
df1 = pd.read_csv('dataset_1.csv')
df1.shape

(50000, 301)

In [ ]:
y = np.array(df1['target'])
X = df1.drop(['target'],axis=1)

Sprawdźmy jak rozkłada się zmienna target

In [ ]:
unique, counts = np.unique(y, return_counts=True)
print(dict(zip(unique, counts)))

{0: 48009, 1: 1991}


Widzimy, że zdecydowanie więcej jest 0 niż 1

Podział na zbiory test, train i val

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y, stratify=y, test_size=0.3, random_state=42
)
X_val, X_test, y_val, y_test = train_test_split(
    X_val, y_val, stratify=y_val, test_size=0.3, random_state=42
)

In [ ]:
X_train.shape

(35000, 300)

Tworzymy ramke, która jest kopią X_train, na której będziemy wykonywać różne procedury processingu danych.

In [ ]:
X_train_processing = X_train.copy()

Zaczniemy od usunięcia kolumn ze stałymi wartościami

In [ ]:
constant_features = get_constant_features(X_train_processing)

In [ ]:
constant_features_list = constant_features.query("Desc=='Constant'")['Var'].to_list()
print(constant_features_list)

['var_167', 'var_158', 'var_113', 'var_112', 'var_201', 'var_248', 'var_44', 'var_99', 'var_97', 'var_212', 'var_215', 'var_92', 'var_89', 'var_61', 'var_227', 'var_87', 'var_66', 'var_67', 'var_225', 'var_69', 'var_81', 'var_196', 'var_33', 'var_195', 'var_287', 'var_297', 'var_294', 'var_171', 'var_178', 'var_180', 'var_135', 'var_182', 'var_120', 'var_14', 'var_23', 'var_129', 'var_127', 'var_122', 'var_80']


In [ ]:
print('Rozmiar ramki przed usunięciem stałych kolumn: ', X_train_processing.shape)
X_train_processing.drop(columns = constant_features_list, inplace=True)
print('Rozmiar ramki po usunięciu stałych kolumn: ', X_train_processing.shape)

Rozmiar ramki przed usunięciem stałych kolumn:  (35000, 300)
Rozmiar ramki po usunięciu stałych kolumn:  (35000, 261)


In [ ]:
def drop_constant_features(df):
  constant_features = get_constant_features(df)
  constant_features_list = constant_features.query("Desc=='Constant'")['Var'].to_list()
  df.drop(columns = constant_features_list, inplace=True)
  return df

Zajmiemy się też usunięciem "prawie stałych" kolumn, ustawiony threshold jest na tyle wysoki, że usuwamy jedynie kolumny które mają dosłownie kilka odmiennych wartości. Małe ryzyko utraty informacji

In [ ]:
constant_features = get_constant_features(X_train_processing, threshold=0.9999, dropna=False)

In [ ]:
quasi_constant_features_list = constant_features.query("Desc=='Quasi Constant'")['Var'].to_list()
print(quasi_constant_features_list)

['var_2', 'var_36', 'var_133', 'var_124', 'var_104', 'var_73', 'var_223', 'var_170', 'var_183', 'var_34', 'var_234', 'var_243', 'var_247', 'var_280', 'var_283', 'var_233', 'var_217', 'var_210', 'var_285', 'var_189', 'var_187', 'var_228', 'var_150', 'var_153', 'var_151', 'var_10', 'var_11', 'var_12', 'var_28', 'var_65', 'var_72', 'var_111', 'var_141', 'var_6', 'var_71', 'var_116', 'var_265', 'var_9', 'var_7', 'var_289']


Jak widać jest ich całkiem sporo

In [ ]:
print('Rozmiar ramki przed usunięciem "prawie stałych" kolumn:  ', X_train_processing.shape)
X_train_processing.drop(columns = quasi_constant_features_list, inplace=True)
print('Rozmiar ramki po usunięciu "prawie stałych" kolumn: ', X_train_processing.shape)

Rozmiar ramki przed usunięciem "prawie stałych" kolumn:   (35000, 261)
Rozmiar ramki po usunięciu "prawie stałych" kolumn:  (35000, 221)


In [ ]:
def drop_quasi_constant_features(df):
  quasi_constant_features = get_constant_features(X_train, threshold=0.9999, dropna=False)
  quasi_constant_features_list = quasi_constant_features.query("Desc=='Quasi Constant'")['Var'].to_list()
  df.drop(columns = quasi_constant_features_list, inplace=True)
  return df

Usunięcie zduplikowanych kolumn

In [ ]:
duplicate_features = get_duplicate_features(X_train_processing)

In [ ]:
duplicate_features_list = duplicate_features.query("Desc=='Duplicate Values'")['feature2'].to_list()
print(duplicate_features_list)

['var_239', 'var_148', 'var_269', 'var_232', 'var_106', 'var_216', 'var_263', 'var_250', 'var_199', 'var_296']


In [ ]:
print('Rozmiar ramki przed usunięciem zduplikowanych kolumn:  ', X_train_processing.shape)
X_train_processing.drop(columns = duplicate_features_list, inplace = True)
print('Rozmiar ramki po usunięciu zduplikowanych kolumn: ', X_train_processing.shape)


Rozmiar ramki przed usunięciem zduplikowanych kolumn:   (35000, 221)
Rozmiar ramki po usunięciu zduplikowanych kolumn:  (35000, 211)


In [ ]:
def drop_duplicates(df):
  duplicate_features = get_duplicate_features(df)
  duplicate_features_list = duplicate_features.query("Desc=='Duplicate Values'")['feature2'].to_list()
  df.drop(columns = duplicate_features_list, inplace = True)
  return df

Outliery

In [ ]:
Q1 = X_train.quantile(0.25)
Q3 = X_train.quantile(0.75)
IQR = Q3 - Q1
print(((X_train < (Q1 - 1.5 * IQR)) | (X_train > (Q3 + 1.5 * IQR))).sum())
print("W sumie wszystkich outlierów jest: ", ((X_train < (Q1 - 1.5 * IQR)) | (X_train > (Q3 + 1.5 * IQR))).sum().sum())

var_1        16
var_2         1
var_3        16
var_4      2863
var_5       193
           ... 
var_296     841
var_297       0
var_298      63
var_299      25
var_300    1341
Length: 300, dtype: int64
W sumie wszystkich outlierów jest:  326676


Spróbujemy dobrać inne granice klasyfikacji outlierów i zobaczyć czy nadal jest ich tak dużo. Zmieńmy współczynnik przy IQR nas 2.5

In [ ]:
Q1 = X_train.quantile(0.25)
Q3 = X_train.quantile(0.75)
IQR = Q3 - Q1
print(((X_train < (Q1 - 2.5 * IQR)) | (X_train > (Q3 + 2.5 * IQR))).sum())
print("W sumie wszystkich outlierów jest: ", ((X_train < (Q1 - 2.5 * IQR)) | (X_train > (Q3 + 2.5 * IQR))).sum().sum())

var_1        16
var_2         1
var_3        16
var_4       775
var_5       193
           ... 
var_296     841
var_297       0
var_298      63
var_299      25
var_300    1341
Length: 300, dtype: int64
W sumie wszystkich outlierów jest:  309035


Nadal outlierów jest bardzo dużo, dlatego usuwanie nie wchodzi w grę. Uważamy, że najlepszym sposobem będzie przeskalowanie wszystkich wartości z użyciem Robust Scaling'u, który idealnie nadaje się do przypadków, w których mamy do czynienia z dużą ilością outlierów i chcemy przeskalować wartości i nie chcemy, żeby outliery "przyćmiły" resztę punktów.

In [ ]:
robust_scaler = RobustScaler()
robust_scaler.fit(X_train_processing)
scaled_df = pd.DataFrame(robust_scaler.fit_transform(X_train_processing), columns = X_train_processing.columns)
scaled_df.reset_index(drop=True)
scaled_df

,var_1,var_3,var_4,var_5,var_8,var_13,var_15,var_16,var_17,var_18,...,var_286,var_288,var_290,var_291,var_292,var_293,var_295,var_298,var_299,var_300
0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0000,0.00,0.0000,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00
1,0.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0000,0.00,0.0000,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00
2,0.0,0.0,-0.01,0.0,0.0,0.0,0.0,0.0000,14.25,0.0000,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00
3,0.0,0.0,0.99,0.0,0.0,0.0,0.0,0.0000,0.00,0.0000,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00
4,0.0,0.0,-0.95,0.0,0.0,0.0,-1.0,0.0000,0.00,0.0000,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34995,0.0,0.0,2.73,0.0,0.0,0.0,-1.0,0.0000,5.82,31530.0672,...,0.0,0.0,0.0,0.0,8.55,0.0,0.0,0.0,0.0,0.00
34996,0.0,0.0,0.95,0.0,0.0,0.0,0.0,0.0000,0.00,0.0000,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00
34997,0.0,0.0,-0.95,0.0,0.0,0.0,-1.0,0.0000,5.82,0.0000,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00
34998,0.0,0.0,1.96,0.0,0.0,0.0,0.0,7667.4717,0.00,0.0000,...,0.0,0.0,0.0,3.0,0.00,0.0,3.0,0.0,0.0,891.36


In [ ]:
X_train_processing = scaled_df

Przejdziemy teraz do różnych opcji dalszego usuwania kolumn

1. Usunięcie kolumn, które mają 99% lub więcej takich samych wartości, w przeciwieństwie do 99,99% zrobionego wcześniej.


In [ ]:
X_1 = X_train_processing.copy()
constant_features = get_constant_features(X_1, threshold=0.99, dropna=False)
quasi_constant_features_list1 = constant_features.query("Desc=='Quasi Constant'")['Var'].to_list()
print(quasi_constant_features_list1)

['var_267', 'var_20', 'var_221', 'var_146', 'var_257', 'var_59', 'var_126', 'var_90', 'var_149', 'var_204', 'var_274', 'var_136', 'var_3', 'var_1', 'var_202', 'var_138', 'var_95', 'var_264', 'var_60', 'var_235', 'var_184', 'var_42', 'var_290', 'var_237', 'var_77', 'var_142', 'var_260', 'var_254', 'var_299', 'var_45', 'var_53', 'var_78', 'var_219', 'var_211', 'var_43', 'var_224', 'var_246', 'var_197', 'var_32', 'var_102', 'var_24', 'var_48', 'var_125', 'var_115', 'var_98', 'var_282', 'var_159', 'var_286', 'var_39', 'var_298', 'var_245', 'var_249', 'var_236', 'var_40', 'var_16', 'var_130', 'var_56', 'var_251', 'var_291', 'var_238', 'var_194', 'var_193', 'var_128', 'var_293', 'var_84', 'var_119', 'var_177', 'var_139', 'var_13', 'var_94', 'var_198', 'var_278', 'var_256', 'var_273', 'var_137', 'var_22', 'var_26', 'var_156', 'var_242', 'var_240', 'var_218', 'var_5', 'var_25', 'var_63', 'var_134', 'var_268', 'var_27', 'var_165', 'var_213', 'var_68']


In [ ]:
print('Rozmiar ramki przed usunięciem "prawie stałych" kolumn: ', X_1.shape)
X_1.drop(columns = quasi_constant_features_list1, inplace=True)
print('Rozmiar ramki po usunięciu "prawie stałych" kolumn: ', X_1.shape)

Rozmiar ramki przed usunięciem "prawie stałych" kolumn:  (35000, 211)
Rozmiar ramki po usunięciu "prawie stałych" kolumn:  (35000, 121)


In [ ]:
def drop_quasi_constant_features_99_percent(df):
  constant_features = get_constant_features(df, threshold=0.99, dropna=False)
  quasi_constant_features_list = constant_features.query("Desc=='Quasi Constant'")['Var'].to_list()
  df.drop(columns = quasi_constant_features_list, inplace=True)
  return df

2. Usuwanie (jednej z) kolumn, których współczynnik korelacji jest większy niż 0.9

Macierz korelacji

In [ ]:
X_2 = X_train_processing.copy()
corr_matrix = X_2.corr()

In [ ]:
corr_matrix

,var_1,var_3,var_4,var_5,var_8,var_13,var_15,var_16,var_17,var_18,...,var_286,var_288,var_290,var_291,var_292,var_293,var_295,var_298,var_299,var_300
var_1,1.000000,0.630636,0.037132,-0.001125,0.041245,-0.000490,0.002604,0.297276,0.027364,0.021827,...,-0.000480,0.059706,-0.000384,0.442369,0.000558,-0.000936,0.003315,-0.000793,-0.000312,0.001927
var_3,0.630636,1.000000,0.020804,-0.000672,0.027813,-0.000293,-0.001581,0.323204,0.024018,0.019813,...,-0.000287,0.015379,-0.000229,0.245379,-0.000075,-0.000559,-0.001376,-0.000474,-0.000186,-0.000247
var_4,0.037132,0.020804,1.000000,0.055579,0.181697,0.063195,0.497553,0.051124,0.322697,0.139324,...,0.050801,0.204327,0.017841,0.095296,0.123425,0.086974,0.184913,0.084464,0.025463,0.086250
var_5,-0.001125,-0.000672,0.055579,1.000000,0.010347,-0.001062,-0.006403,-0.001624,0.017101,0.012274,...,-0.000999,0.006749,-0.001118,-0.002507,-0.003473,0.042068,0.013079,-0.000173,-0.000908,0.008484
var_8,0.041245,0.027813,0.181697,0.010347,1.000000,0.177314,-0.038309,0.020895,0.127774,0.107833,...,0.016503,0.103358,0.009784,0.023629,0.025604,0.068158,0.098834,0.002314,0.004382,0.061480
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
var_293,-0.000936,-0.000559,0.086974,0.042068,0.068158,0.026211,0.018787,0.001377,0.039064,0.043733,...,0.008181,0.032627,0.008685,0.006225,0.001735,1.000000,0.027773,0.007811,0.017819,0.019227
var_295,0.003315,-0.001376,0.184913,0.013079,0.098834,0.014804,-0.067818,0.011224,0.237349,0.009399,...,0.010209,0.229868,0.002840,0.028529,-0.006283,0.027773,1.000000,0.006799,0.005924,0.569761
var_298,-0.000793,-0.000474,0.084464,-0.000173,0.002314,-0.001005,-0.000074,0.005113,0.027486,-0.001800,...,0.580241,0.000794,0.016537,0.026981,-0.003055,0.007811,0.006799,1.000000,0.034213,0.023733
var_299,-0.000312,-0.000186,0.025463,-0.000908,0.004382,-0.000396,-0.016162,0.008481,0.006651,0.000473,...,0.030205,0.007729,0.880781,0.026921,-0.001203,0.017819,0.005924,0.034213,1.000000,0.002191


Przechodzimy po dolnym trójkącie macierzy korelacji i zapisujemy w tablicy numery jednej z dwóch kolumn, których korelacja jest wyższa lub równa 0.9.

In [ ]:
n = X_2.shape[1]
threshold = 0.90
var_to_drop = []
for j in range(n):
  for i in range(j):
    if corr_matrix.iloc[i][j] >= threshold:
      var_to_drop.append(corr_matrix.index[i])

In [ ]:
var_to_drop = set(var_to_drop)
var_to_drop = list(var_to_drop)
print("liczba kolumn które usuwamy:" ,len(var_to_drop))
print("lista kolumn, które usuwamy:", var_to_drop)

liczba kolumn które usuwamy: 78
lista kolumn, które usuwamy: ['var_45', 'var_78', 'var_236', 'var_165', 'var_39', 'var_31', 'var_118', 'var_123', 'var_88', 'var_194', 'var_53', 'var_102', 'var_245', 'var_284', 'var_168', 'var_164', 'var_98', 'var_144', 'var_83', 'var_121', 'var_145', 'var_162', 'var_86', 'var_138', 'var_59', 'var_95', 'var_15', 'var_58', 'var_137', 'var_56', 'var_282', 'var_108', 'var_47', 'var_197', 'var_140', 'var_207', 'var_27', 'var_103', 'var_84', 'var_177', 'var_251', 'var_51', 'var_96', 'var_57', 'var_30', 'var_169', 'var_60', 'var_244', 'var_8', 'var_175', 'var_54', 'var_155', 'var_19', 'var_91', 'var_41', 'var_117', 'var_253', 'var_160', 'var_18', 'var_70', 'var_249', 'var_94', 'var_43', 'var_220', 'var_4', 'var_16', 'var_222', 'var_40', 'var_229', 'var_50', 'var_52', 'var_37', 'var_105', 'var_77', 'var_38', 'var_100', 'var_176', 'var_277']


In [ ]:
X_2 = X_2.drop(var_to_drop, axis = 1)
X_2.shape

(35000, 133)

In [ ]:
def variance_drop(df):
  corr_matrix = df.corr()

  n = df.shape[1]
  threshold = 0.90
  var_to_drop = []
  for j in range(n):
    for i in range(j):
      if corr_matrix.iloc[i][j] >= threshold:
        var_to_drop.append(corr_matrix.index[i])
  
  var_to_drop = set(var_to_drop)
  var_to_drop = list(var_to_drop)

  df = df.drop(var_to_drop, axis = 1)
  return df

Usunięcie constant features, quasi contant features oraz duplicate features, które uważamy za niezbędne wykonamy najpierw, a potem z tego wstępnie preprocesowanego zbioru będziemy tworzyć różne bardziej z procesowane zbiory i oddzielnie sprawdzać różne metody

In [ ]:
def base_preprocessing(df):
  df_out = df.copy()
  df_out = drop_constant_features(df_out)
  df_out = drop_quasi_constant_features(df_out)
  df_out = drop_duplicates(df_out)
  

  robust_scaler = RobustScaler()
  robust_scaler.fit(df_out)
  scaled_df = pd.DataFrame(robust_scaler.fit_transform(df_out), columns = df_out.columns)
  scaled_df.reset_index(drop=True)
  df_out = scaled_df
  return df_out

In [ ]:
X_preprocessed = base_preprocessing(X_train)
print("Rozmiar ramki po podstawowym preprocessingu: ",X_preprocessed.shape)

Rozmiar ramki po podstawowym preprocessingu:  (35000, 211)


In [ ]:
def preprocessing1(df):
  df_out = df.copy()
  df_out = drop_quasi_constant_features_99_percent(df_out)
  return df_out

In [ ]:
X_preprocessed1 = preprocessing1(X_preprocessed)
print("Wymiary zbioru po preprocessingu 1 tj. usunięciu kolumn z 99% zgodnością wartości: ",X_preprocessed1.shape)

Wymiary zbioru po preprocessingu 1 tj. usunięciu kolumn z 99% zgodnością wartości:  (35000, 121)


In [ ]:
def preprocessing2(df):
  df_out = df.copy()
  df_out = variance_drop(df_out)
  return df_out

In [ ]:
X_preprocessed2 = preprocessing2(X_preprocessed)
print("Wymiary zbioru po preprocessingu 2 tj. usunięciu kolumn z korelacją powyżej 90% : ",X_preprocessed2.shape)

Wymiary zbioru po preprocessingu 2 tj. usunięciu kolumn z korelacją powyżej 90% :  (35000, 133)


In [ ]:
def preprocessing3(df):
  pass
  #zrób tu to pca, ja wole tobie to zostawić

In [ ]:
def preprocessing1_2(df):
  df_out = df.copy()
  df_out = base_preprocessing(df_out)
  df_out = preprocessing1(df_out)
  df_out = preprocessing2(df_out)
  return df_out

In [ ]:
X_preprocessed1_2 = preprocessing1_2(X_train)
print("Wymiary zbioru po preprocessingu 1 oraz preprocesingu 2 tj. usunięciu kolumn z 99% zgodnością wartości oraz usunięciu kolumn z korelacją powyżej 90% : ",X_preprocessed1_2.shape)

Wymiary zbioru po preprocessingu 1 oraz preprocesingu 2 tj. usunięciu kolumn z 99% zgodnością wartości oraz usunięciu kolumn z korelacją powyżej 90% :  (35000, 71)


Import modułu do liczenia celności oraz precyzji modelu

In [ ]:
from sklearn.metrics import precision_score, accuracy_score

Model używający liniowej regresji

In [ ]:
from sklearn import linear_model
lr_model = linear_model.LogisticRegression()

Base preprocessing

In [ ]:
lr_model.fit(X_preprocessed, y_train)
X_test_base = X_test[X_test.columns.intersection(X_preprocessed.columns)]
print("Rozmiar ramki: ", X_test_base.shape)

y_pred = lr_model.predict(X_test_base)
unique, counts = np.unique(y_pred, return_counts=True)
print("Rozkład y_pred: ", dict(zip(unique, counts)))

acc = (y_pred == y_test)
print("Accuracy: ", sum(acc)/len(y_test))
print("Precision: ", precision_score(y_test, y_pred))

Rozmiar ramki:  (4500, 211)
Rozkład y_pred:  {0: 4348, 1: 152}
accuracy:  0.9317777777777778
precision 0.07894736842105263


Preprocessing 1

In [ ]:
lr_model.fit(X_preprocessed1, y_train)
X_test_1 = X_test[X_test.columns.intersection(X_preprocessed1.columns)]
print("Rozmiar ramki: ", X_test_1.shape)

y_pred = lr_model.predict(X_test_1)
unique, counts = np.unique(y_pred, return_counts=True)
print("Rozkład y_pred: ", dict(zip(unique, counts)))

acc = (y_pred == y_test)
print("Accuracy: ", sum(acc)/len(y_test))
print("Precision: ", precision_score(y_test, y_pred))

Rozmiar ramki:  (4500, 121)
Rozkład y_pred:  {0: 4500}
accuracy:  0.9602222222222222
precision 0.0


Preprocessing 2

In [ ]:
lr_model.fit(X_preprocessed2, y_train)
X_test_2 = X_test[X_test.columns.intersection(X_preprocessed2.columns)]
print("Rozmiar ramki: ",X_test_2.shape)

y_pred = lr_model.predict(X_test_2)
unique, counts = np.unique(y_pred, return_counts=True)
print("Rozkład y_pred: ", dict(zip(unique, counts)))

acc = (y_pred == y_test)
print("Accuracy: ",sum(acc)/len(y_test))
print("Precision: ", precision_score(y_test, y_pred))

Rozmiar ramki:  (4500, 133)
Rozkład y_pred:  {0: 4500}
accuracy:  0.9602222222222222
precision 0.0


Preprocessing 1 i 2

In [ ]:
lr_model.fit(X_preprocessed1_2, y_train)
X_test1_2 = X_test[X_test.columns.intersection(X_preprocessed1_2.columns)]
print("Rozmiar ramki: ",X_test1_2.shape)

y_pred = lr_model.predict(X_test1_2)
unique, counts = np.unique(y_pred, return_counts=True)
print("Rozkład y_pred: ", dict(zip(unique, counts)))

acc = (y_pred == y_test)
print("Accuracy: ",sum(acc)/len(y_test))
print("Precision: ", precision_score(y_test, y_pred))

Rozmiar ramki:  (4500, 71)
Rozkład y_pred:  {0: 4500}
accuracy:  0.9602222222222222
precision 0.0


Model SVM

In [ ]:
from sklearn import svm

clf = svm.SVC() 

Base Preprocessing

Preprocessing 1

In [ ]:
clf.fit(X_preprocessed1, y_train)
X_test_1 = X_test[X_test.columns.intersection(X_preprocessed1.columns)]
print("Rozmiar ramki :", X_test_1.shape)

y_pred = clf.predict(X_test_1)
unique, counts = np.unique(y_pred, return_counts=True)
print("Rozkład y_pred :", dict(zip(unique, counts)))

print("Accuracy: ",accuracy_score(y_test, y_pred))
print("Precision: ", precision_score(y_test, y_pred))


Rozmiar ramki : (4500, 121)
Rozkład y_pred : {0: 4500}
Accuracy: 0.9602222222222222
precision 0.0


Preprocessing 2

In [ ]:
clf.fit(X_preprocessed2, y_train)
X_test_2 = X_test[X_test.columns.intersection(X_preprocessed2.columns)]
print("Rozmiar ramki :", X_test_2.shape)

y_pred = clf.predict(X_test_2)
unique, counts = np.unique(y_pred, return_counts=True)
print("Rozkład y_pred :", dict(zip(unique, counts)))

print("Accuracy: ", accuracy_score(y_test, y_pred))
print("Precision: ", precision_score(y_test, y_pred))

Rozmiar ramki : (4500, 133)
Rozkład y_pred : {0: 4500}
Accuracy: 0.9602222222222222
Precision 0.0


Preprocessing 1 i 2

In [ ]:
clf.fit(X_preprocessed1_2, y_train)
X_test1_2 = X_test[X_test.columns.intersection(X_preprocessed1_2.columns)]
print("Rozmiar ramki :", X_test1_2.shape)

y_pred = clf.predict(X_test1_2)
unique, counts = np.unique(y_pred, return_counts=True)
print("Rozkład y_pred: ", dict(zip(unique, counts)))

print("Accuracy: ", accuracy_score(y_test, y_pred))
print("Precision: ", precision_score(y_test, y_pred))

Rozmiar ramki : (4500, 71)
Rozkład y_pred : {0: 4500}
Accuracy: 0.9602222222222222
Precision 0.0


Regresja Random Tree Forest

n_estimators = 280

In [ ]:
regressor = RandomForestRegressor(n_estimators = 280, random_state = 0)

Base Preprocessing

In [ ]:
regressor.fit(X_preprocessed, y_train)
X_test_base = X_test[X_test.columns.intersection(X_preprocessed.columns)]
print("Rozmiar ramki: ", X_test_base.shape)

y_pred = regressor.predict(X_test_base)
y_pred = np.round(y_pred)
unique, counts = np.unique(y_pred, return_counts=True)
print("Rozkład y_pred: ", dict(zip(unique, counts)))

acc = (y_pred == y_test)
print("Accuracy: ",sum(acc)/len(y_test))
print("Precision: ", precision_score(y_test, y_pred))

Rozmiar ramki:  (4500, 211)
Rozkład y_pred : {0.0: 4481, 1.0: 19}
accuracy:  0.9564444444444444
precision 0.05263157894736842


Preprocessing 1

In [ ]:
regressor.fit(X_preprocessed1, y_train)
X_test_1 = X_test[X_test.columns.intersection(X_preprocessed1.columns)]
print("Rozmiar ramki: ", X_test_1.shape)

y_pred = regressor.predict(X_test_1)
y_pred = np.round(y_pred)
unique, counts = np.unique(y_pred, return_counts=True)
print("Rozkład y_pred: ", dict(zip(unique, counts)))

acc = (y_pred == y_test)
print("Accuracy: ",sum(acc)/len(y_test))
print("Precision: ", precision_score(y_test, y_pred))

Rozmiar ramki : (4500, 121)
Rozkład y_pred : {0.0: 4484, 1.0: 16}
accuracy:  0.9571111111111111
precision 0.0625


Preprocessing 2

In [ ]:
regressor.fit(X_preprocessed2, y_train)
X_test_2 = X_test[X_test.columns.intersection(X_preprocessed2.columns)]
print("Rozmiar ramki: ", X_test_2.shape)

y_pred = regressor.predict(X_test_2)
y_pred = np.round(y_pred)
unique, counts = np.unique(y_pred, return_counts=True)
print("Rozkład y_pred: ", dict(zip(unique, counts)))

acc = (y_pred == y_test)
print("Accuracy: ",sum(acc)/len(y_test))
print("Precision: ", precision_score(y_test, y_pred))

Rozmiar ramki : (4500, 133)
Rozkład y_pred : {0.0: 4483, 1.0: 17}
accuracy:  0.9568888888888889
precision 0.058823529411764705


Preprocessing 1 i 2

In [ ]:
regressor.fit(X_preprocessed1_2, y_train)
X_test1_2 = X_test[X_test.columns.intersection(X_preprocessed1_2.columns)]
print("Rozmiar ramki: ", X_test1_2.shape)

y_pred = regressor.predict(X_test1_2)
y_pred = np.round(y_pred)
unique, counts = np.unique(y_pred, return_counts=True)
print("Rozkład y_pred: ", dict(zip(unique, counts)))

acc = (y_pred == y_test)
print("Accuracy: ",sum(acc)/len(y_test))
print("Precision: ", precision_score(y_test, y_pred))

Rozmiar ramki : (4500, 71)
Rozkład y_pred : {0.0: 4486, 1.0: 14}
accuracy:  0.9575555555555556
precision 0.07142857142857142


n_estimators = 210



In [ ]:
regressor = RandomForestRegressor(n_estimators = 210, random_state = 0)

Preprocessing 1

In [ ]:
regressor.fit(X_preprocessed1, y_train)
X_test_1 = X_test[X_test.columns.intersection(X_preprocessed1.columns)]
print("Rozmiar ramki: ", X_test_1.shape)

y_pred = regressor.predict(X_test_1)
y_pred = np.round(y_pred)
unique, counts = np.unique(y_pred, return_counts=True)
print("Rozkład y_pred: ", dict(zip(unique, counts)))

acc = (y_pred == y_test)
print("Accuracy: ",sum(acc)/len(y_test))
print("Precision: ", precision_score(y_test, y_pred))

Rozmiar ramki:  (4500, 121)
Rozkład y_pred:  {0.0: 4485, 1.0: 15}
Accuracy:  0.9573333333333334
Precision:  0.06666666666666667


Preprocessing 2

In [ ]:
regressor.fit(X_preprocessed2, y_train)
X_test_2 = X_test[X_test.columns.intersection(X_preprocessed2.columns)]
print("Rozmiar ramki: ", X_test_2.shape)

y_pred = regressor.predict(X_test_2)
y_pred = np.round(y_pred)
unique, counts = np.unique(y_pred, return_counts=True)
print("Rozkład y_pred: ", dict(zip(unique, counts)))

acc = (y_pred == y_test)
print("Accuracy: ",sum(acc)/len(y_test))
print("Precision: ", precision_score(y_test, y_pred))

Rozmiar ramki:  (4500, 133)
Rozkład y_pred:  {0.0: 4485, 1.0: 15}
Accuracy:  0.9573333333333334
Precision:  0.06666666666666667


Preprocessing 1 i 2

In [ ]:
regressor.fit(X_preprocessed1_2, y_train)
X_test1_2 = X_test[X_test.columns.intersection(X_preprocessed1_2.columns)]
print("Rozmiar ramki: ", X_test1_2.shape)

y_pred = regressor.predict(X_test1_2)
y_pred = np.round(y_pred)
unique, counts = np.unique(y_pred, return_counts=True)
print("Rozkład y_pred: ", dict(zip(unique, counts)))

acc = (y_pred == y_test)
print("Accuracy: ",sum(acc)/len(y_test))
print("Precision: ", precision_score(y_test, y_pred))

Rozmiar ramki:  (4500, 71)
Rozkład y_pred:  {0.0: 4487, 1.0: 13}
Accuracy:  0.9577777777777777
Precision:  0.07692307692307693


n_estimators = 140

In [ ]:
regressor = RandomForestRegressor(n_estimators = 140, random_state = 0)

Preprocessing 1

In [ ]:
regressor.fit(X_preprocessed1, y_train)
X_test_1 = X_test[X_test.columns.intersection(X_preprocessed1.columns)]
print("Rozmiar ramki: ", X_test_1.shape)

y_pred = regressor.predict(X_test_1)
y_pred = np.round(y_pred)
unique, counts = np.unique(y_pred, return_counts=True)
print("Rozkład y_pred: ", dict(zip(unique, counts)))

acc = (y_pred == y_test)
print("Accuracy: ",sum(acc)/len(y_test))
print("Precision: ", precision_score(y_test, y_pred))

Rozmiar ramki:  (4500, 121)
Rozkład y_pred:  {0.0: 4482, 1.0: 18}
Accuracy:  0.9566666666666667
Precision:  0.05555555555555555


Preprocessing 2

In [ ]:
regressor.fit(X_preprocessed2, y_train)
X_test_2 = X_test[X_test.columns.intersection(X_preprocessed2.columns)]
print("Rozmiar ramki: ", X_test_2.shape)

y_pred = regressor.predict(X_test_2)
y_pred = np.round(y_pred)
unique, counts = np.unique(y_pred, return_counts=True)
print("Rozkład y_pred: ", dict(zip(unique, counts)))

acc = (y_pred == y_test)
print("Accuracy: ",sum(acc)/len(y_test))
print("Precision: ", precision_score(y_test, y_pred))

Rozmiar ramki:  (4500, 133)
Rozkład y_pred:  {0.0: 4489, 1.0: 11}
Accuracy:  0.9582222222222222
Precision:  0.09090909090909091


Preprocessing 1 i 2

In [ ]:
regressor.fit(X_preprocessed1_2, y_train)
X_test1_2 = X_test[X_test.columns.intersection(X_preprocessed1_2.columns)]
print("Rozmiar ramki: ", X_test1_2.shape)

y_pred = regressor.predict(X_test1_2)
y_pred = np.round(y_pred)
unique, counts = np.unique(y_pred, return_counts=True)
print("Rozkład y_pred: ", dict(zip(unique, counts)))

acc = (y_pred == y_test)
print("Accuracy: ",sum(acc)/len(y_test))
print("Precision: ", precision_score(y_test, y_pred))

Rozmiar ramki:  (4500, 71)
Rozkład y_pred:  {0.0: 4488, 1.0: 12}
Accuracy:  0.958
Precision:  0.08333333333333333


n_estimators = 100

In [ ]:
regressor = RandomForestRegressor(n_estimators = 100, random_state = 0)

Preprocessing 1

In [ ]:
regressor.fit(X_preprocessed1, y_train)
X_test_1 = X_test[X_test.columns.intersection(X_preprocessed1.columns)]
print("Rozmiar ramki: ", X_test_1.shape)

y_pred = regressor.predict(X_test_1)
y_pred = np.round(y_pred)
unique, counts = np.unique(y_pred, return_counts=True)
print("Rozkład y_pred: ", dict(zip(unique, counts)))

acc = (y_pred == y_test)
print("Accuracy: ",sum(acc)/len(y_test))
print("Precision: ", precision_score(y_test, y_pred))

Rozmiar ramki:  (4500, 121)
Rozkład y_pred:  {0.0: 4475, 1.0: 25}
Accuracy:  0.9551111111111111
Precision:  0.04


Preprocessing 2

In [ ]:
regressor.fit(X_preprocessed2, y_train)
X_test_2 = X_test[X_test.columns.intersection(X_preprocessed2.columns)]
print("Rozmiar ramki: ", X_test_2.shape)

y_pred = regressor.predict(X_test_2)
y_pred = np.round(y_pred)
unique, counts = np.unique(y_pred, return_counts=True)
print("Rozkład y_pred: ", dict(zip(unique, counts)))

acc = (y_pred == y_test)
print("Accuracy: ",sum(acc)/len(y_test))
print("Precision: ", precision_score(y_test, y_pred))

Rozmiar ramki:  (4500, 133)
Rozkład y_pred:  {0.0: 4487, 1.0: 13}
Accuracy:  0.9577777777777777
Precision:  0.07692307692307693


Preprocessing 1 i 2

In [ ]:
regressor.fit(X_preprocessed1_2, y_train)
X_test1_2 = X_test[X_test.columns.intersection(X_preprocessed1_2.columns)]
print("Rozmiar ramki: ", X_test1_2.shape)

y_pred = regressor.predict(X_test1_2)
y_pred = np.round(y_pred)
unique, counts = np.unique(y_pred, return_counts=True)
print("Rozkład y_pred: ", dict(zip(unique, counts)))

acc = (y_pred == y_test)
print("Accuracy: ",sum(acc)/len(y_test))
print("Precision: ", precision_score(y_test, y_pred))

Rozmiar ramki:  (4500, 71)
Rozkład y_pred:  {0.0: 4488, 1.0: 12}
Accuracy:  0.958
Precision:  0.08333333333333333


Random forest po L1

n_estimators = 100

In [ ]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression

In [ ]:
sel = SelectFromModel(LogisticRegression(penalty = "l1", C = 1, solver = "liblinear"))

Base preprocessing

In [ ]:
sel.fit(X_preprocessed, y_train)

SelectFromModel(estimator=LogisticRegression(C=1, penalty='l1',
                                             solver='liblinear'))

In [ ]:
X_preprocessed_l1 = sel.transform(X_preprocessed)
X_preprocessed_l1 = pd.DataFrame(X_preprocessed_l1)
X_test_l1 = X_test[X_test.columns.intersection(X_preprocessed.columns)]
X_test_l1 = sel.transform(X_test_l1)

In [ ]:
regressor.fit(X_preprocessed_l1, y_train)
print("Rozmiar ramki :", X_test_l1.shape)

y_pred = regressor.predict(X_test_l1)
y_pred = np.round(y_pred)
unique, counts = np.unique(y_pred, return_counts=True)
print("Rozkład y_pred :", dict(zip(unique, counts)))

acc = (y_pred == y_test)
print("accuracy: ",sum(acc)/len(y_test))
print("precision", precision_score(y_test, y_pred))

Rozmiar ramki : (4500, 124)
Rozkład y_pred : {0.0: 4485, 1.0: 15}
accuracy:  0.9573333333333334
precision 0.06666666666666667


Preprocessing 1

In [ ]:
sel.fit(X_preprocessed1, y_train)

SelectFromModel(estimator=LogisticRegression(C=1, penalty='l1',
                                             solver='liblinear'))

In [ ]:
X_preprocessed_l1 = sel.transform(X_preprocessed1)
X_preprocessed_l1 = pd.DataFrame(X_preprocessed_l1)
X_test_l1 = X_test[X_test.columns.intersection(X_preprocessed1.columns)]
sel.get_support()
X_test_l1 = sel.transform(X_test_l1)

In [ ]:
regressor.fit(X_preprocessed_l1, y_train)
print("Rozmiar ramki :", X_test_l1.shape)

y_pred = regressor.predict(X_test_l1)
y_pred = np.round(y_pred)
unique, counts = np.unique(y_pred, return_counts=True)
print("Rozkład y_pred :", dict(zip(unique, counts)))

acc = (y_pred == y_test)
print("accuracy: ",sum(acc)/len(y_test))
print("precision", precision_score(y_test, y_pred))

Rozmiar ramki : (4500, 98)
Rozkład y_pred : {0.0: 4482, 1.0: 18}
accuracy:  0.9566666666666667
precision 0.05555555555555555


Preprocessing 2

In [ ]:
sel.fit(X_preprocessed2, y_train)

SelectFromModel(estimator=LogisticRegression(C=1, penalty='l1',
                                             solver='liblinear'))

In [ ]:
X_preprocessed_l1 = sel.transform(X_preprocessed2)
X_preprocessed_l1 = pd.DataFrame(X_preprocessed_l1)
X_test_l1 = X_test[X_test.columns.intersection(X_preprocessed2.columns)]
X_test_l1 = sel.transform(X_test_l1)

In [ ]:
regressor.fit(X_preprocessed_l1, y_train)
print("Rozmiar ramki :", X_test_l1.shape)

y_pred = regressor.predict(X_test_l1)
y_pred = np.round(y_pred)
unique, counts = np.unique(y_pred, return_counts=True)
print("Rozkład y_pred :", dict(zip(unique, counts)))

acc = (y_pred == y_test)
print("accuracy: ",sum(acc)/len(y_test))
print("precision", precision_score(y_test, y_pred))

Rozmiar ramki : (4500, 88)
Rozkład y_pred : {0.0: 4489, 1.0: 11}
accuracy:  0.9582222222222222
precision 0.09090909090909091


Preprocessing 1 i 2

In [ ]:
sel.fit(X_preprocessed1_2, y_train)

SelectFromModel(estimator=LogisticRegression(C=1, penalty='l1',
                                             solver='liblinear'))

In [ ]:
X_preprocessed_l1 = sel.transform(X_preprocessed1_2)
X_preprocessed_l1 = pd.DataFrame(X_preprocessed_l1)
X_test_l1 = X_test[X_test.columns.intersection(X_preprocessed1_2.columns)]
X_test_l1 = sel.transform(X_test_l1)

In [ ]:
regressor.fit(X_preprocessed_l1, y_train)
print("Rozmiar ramki :", X_test_l1.shape)

y_pred = regressor.predict(X_test_l1)
y_pred = np.round(y_pred)
unique, counts = np.unique(y_pred, return_counts=True)
print("Rozkład y_pred :", dict(zip(unique, counts)))

acc = (y_pred == y_test)
print("accuracy: ",sum(acc)/len(y_test))
print("precision", precision_score(y_test, y_pred))

Rozmiar ramki : (4500, 56)
Rozkład y_pred : {0.0: 4489, 1.0: 11}
accuracy:  0.9582222222222222
precision 0.09090909090909091


n_estimators = 120

In [ ]:
regressor = RandomForestRegressor(n_estimators = 120, random_state = 0)

Preprocessing 2

In [ ]:
sel.fit(X_preprocessed2, y_train)

SelectFromModel(estimator=LogisticRegression(C=1, penalty='l1',
                                             solver='liblinear'))

In [ ]:
X_preprocessed_l1 = sel.transform(X_preprocessed2)
X_preprocessed_l1 = pd.DataFrame(X_preprocessed_l1)
X_test_l1 = X_test[X_test.columns.intersection(X_preprocessed2.columns)]
X_test_l1 = sel.transform(X_test_l1)

In [ ]:
regressor.fit(X_preprocessed_l1, y_train)
print("Rozmiar ramki :", X_test_l1.shape)

y_pred = regressor.predict(X_test_l1)
y_pred = np.round(y_pred)
unique, counts = np.unique(y_pred, return_counts=True)
print("Rozkład y_pred :", dict(zip(unique, counts)))

acc = (y_pred == y_test)
print("accuracy: ",sum(acc)/len(y_test))
print("precision", precision_score(y_test, y_pred))

Rozmiar ramki : (4500, 88)
Rozkład y_pred : {0.0: 4489, 1.0: 11}
accuracy:  0.9582222222222222
precision 0.09090909090909091


Preprocessing 1 i 2

In [ ]:
sel.fit(X_preprocessed1_2, y_train)

In [ ]:
X_preprocessed_l1 = sel.transform(X_preprocessed1_2)
X_preprocessed_l1 = pd.DataFrame(X_preprocessed_l1)
X_test_l1 = X_test[X_test.columns.intersection(X_preprocessed1_2.columns)]
X_test_l1 = sel.transform(X_test_l1)

In [ ]:
regressor.fit(X_preprocessed_l1, y_train)
print("Rozmiar ramki :", X_test_l1.shape)

y_pred = regressor.predict(X_test_l1)
y_pred = np.round(y_pred)
unique, counts = np.unique(y_pred, return_counts=True)
print("Rozkład y_pred :", dict(zip(unique, counts)))

acc = (y_pred == y_test)
print("accuracy: ",sum(acc)/len(y_test))
print("precision", precision_score(y_test, y_pred))

Rozmiar ramki : (4500, 56)
Rozkład y_pred : {0.0: 4490, 1.0: 10}
accuracy:  0.9584444444444444
precision 0.1
